In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing necessary libararies
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import itertools
import collections
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS

Text Cleaning :

In [ ]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"thx"   : "thanks"
}

def remove_contractions(text):
    return contractions[text.lower()] if text.lower() in contractions.keys() else text

# clean dataset
def clean_dataset(text):
    text = text.lower()
    text = re.sub(r'#','', text)
    text = re.sub(r'\&\w*;', '', text)
    text = re.sub(r'\$\w*', '', text)
    text = re.sub(r'https?:\/\/.*\/\w*', '', text)
    text = re.sub(r'\s\s+','', text)
    text = re.sub(r'[ ]{2, }',' ',text)
    text=  re.sub(r'http(\S)+', '',text)
    text=  re.sub(r'http ...', '',text)
    text=  re.sub(r'(RT|rt)[ ]*@[ ]*[\S]+','',text)
    text=  re.sub(r'RT[ ]?@','',text)
    text = re.sub(r'@[\S]+','',text)
    text = re.sub(r'&amp;?', 'and',text)
    text = re.sub(r'&lt;','<',text)
    text = re.sub(r'&gt;','>',text)
    text = re.sub(r'([\w\d]+)([^\w\d ]+)', '\1 \2',text)
    text = re.sub(r'([^\w\d ]+)([\w\d]+)', '\1 \2',text)
    text= ''.join(c for c in text if c <= '\uFFFF') 
    text = text.strip()
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    text = ' '.join(re.sub("[\.\,\!\?\:\;\-\=\/\|\'\(\']", " ", text).split())
    text = text.replace(":"," ")
    text = ' '.join(text.split()) 
    text = re.sub("([^\x00-\x7F])+"," ",text)
    text = ' '.join(re.sub("[^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a]", " ", text).split())
    return text

#remove stop word i.e. the most frequently appeared words 
myOwnStopList=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'what','how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'do', 'should', "should", 'now', 'd', 'm', 'o', 're', 've', 'y', 'ain', "are", 'could', "was",
'would','have','get','got','getting','one','two','still','going']


In [ ]:
corpus=pd.read_csv("/content/drive/MyDrive/DSF trials/summary.csv")

In [ ]:
corpus.head()

,Plot_summary,Genres
0,"Old Major, the old boar on the Manor Farm, ca...","['children', 'fiction', 'satire', 'nonfiction']"
1,"Alex, a teenager living in near-future Englan...","['fiction', 'literature', 'satire', 'utopian']"
2,The text of The Plague is divided into five p...,"['fiction', 'philosophy', 'literature']"
3,The novel posits that space around the Milky ...,"['fiction', 'fantasy']"
4,"The book tells the story of Paul B�umer, a Ge...","['war', 'nonfiction']"


In [ ]:
corpus['clean_plot_summary']=corpus['Plot_summary'].apply(lambda x: remove_contractions(str(x)))

corpus['clean_plot_summary'] =corpus['clean_plot_summary'].apply(lambda x: clean_dataset(x))
corpus['clean_plot_summary'] = corpus['clean_plot_summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (myOwnStopList)]))

In [ ]:
corpus.head()

,Plot_summary,Genres,clean_plot_summary
0,"Old Major, the old boar on the Manor Farm, ca...","['children', 'fiction', 'satire', 'nonfiction']",old old boar manor calls animals farm compares...
1,"Alex, a teenager living in near-future Englan...","['fiction', 'literature', 'satire', 'utopian']",teenager living leads gang nightly orgies rand...
2,The text of The Plague is divided into five p...,"['fiction', 'philosophy', 'literature']",text plague divided five town thousands initia...
3,The novel posits that space around the Milky ...,"['fiction', 'fantasy']",novel posits space around milky way divided co...
4,"The book tells the story of Paul B�umer, a Ge...","['war', 'nonfiction']",book tells story paul german soldier school ge...


In [ ]:
corpus.head(10)

,Plot_summary,Genres,clean_plot_summary
0,"Old Major, the old boar on the Manor Farm, ca...","['children', 'fiction', 'satire', 'nonfiction']",old old boar manor calls animals farm compares...
1,"Alex, a teenager living in near-future Englan...","['fiction', 'literature', 'satire', 'utopian']",teenager living leads gang nightly orgies rand...
2,The text of The Plague is divided into five p...,"['fiction', 'philosophy', 'literature']",text plague divided five town thousands initia...
3,The novel posits that space around the Milky ...,"['fiction', 'fantasy']",novel posits space around milky way divided co...
4,"The book tells the story of Paul B�umer, a Ge...","['war', 'nonfiction']",book tells story paul german soldier school ge...
5,"Ged is a young boy on Gont, one of the larger...","['children', 'fiction', 'biography', 'fantasy']",ged young boy larger islands north archipelago...
6,"Living on Mars, Deckard is acting as a consul...",['fiction'],living deckard acting consultant movie crew fi...
7,Beginning several months after the events in ...,['fiction'],beginning several months events blade deckard ...
8,The book of Job tells the story of an extreme...,['religious'],book job tells story extremely righteous man n...
9,The story is told through the eyes of narrato...,"['fiction', 'literature']",story told eyes narrator james named author ce...


Installing sentence transformer for plotting word embeddings on a unsupervised note

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Forming embeddings of passages

In [ ]:
corpus['passage_embedding']=corpus['clean_plot_summary'].apply(lambda x: model.encode(x))

In [ ]:
len(corpus['passage_embedding_new'][0])

384

Forming embeddings of genre labels

In [ ]:
labels=['mystery', 'religious', 'cookery', 'fiction', 'children', 'war', 'history', 'thriller', 'literature', 'music', 'utopian', 'satire', 'evil', 'environmental', 'comic', 'horror', 'documentary', 'adventure', 'fantasy', 'business', 'romance', 'educational', 'action', 'comedy', 'anthropology', 'philosophy', 'nonfiction', 'biography', 'sorrow']

lb=[]
for i in labels:
  lb.append(model.encode(i))

labels_vectors= np.asarray([l for l in lb])

We are using cosine similarity to check passage<->genre vector similarity as eucledian distance or mse under higher dimensions suffer "curse of dimensionality"

In [ ]:
from scipy import spatial
from sklearn.metrics import mean_squared_error
def sort_index(lst, rev=True):
    index = range(len(lst))
    s = sorted(index, reverse=rev, key=lambda i: lst[i])
    return s

def compute_top_3_cosine_similar_genres(item,label_vector):
      result=[]
      for genre_vector in label_vector:
        result.append(1 - spatial.distance.cosine(item, genre_vector))
      top_3_index=sort_index(result)[:3]
      final_genre=[labels[i] for i in top_3_index]
      return sorted(final_genre) 


        


Computing cosine similarity and taking top 3 closest genres

In [ ]:
corpus['multi_label_genre']=corpus['passage_embedding'].apply(lambda x: compute_top_3_cosine_similar_genres(x,labels_vectors))

Checking output distribution

In [ ]:
corpus['multi_label_genre'].value_counts()

TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[fiction, literature, nonfiction]      1297
[biography, literature, nonfiction]     546
[literature, nonfiction, thriller]      261
[fiction, literature, thriller]         241
[adventure, literature, nonfiction]     213
                                       ... 
[action, comedy, nonfiction]              1
[fantasy, literature, satire]             1
[business, children, history]             1
[business, comedy, thriller]              1
[business, documentary, evil]             1
Name: multi_label_genre, Length: 1728, dtype: int64

In [ ]:
corpus.head(10)

,Plot_summary,Genres,clean_plot_summary,passage_embedding,multi_label_genre
0,"Old Major, the old boar on the Manor Farm, ca...","['children', 'fiction', 'satire', 'nonfiction']",old old boar manor calls animals farm compares...,"[-0.00936466, -0.046432283, 0.04785304, -0.004...","[action, cookery, war]"
1,"Alex, a teenager living in near-future Englan...","['fiction', 'literature', 'satire', 'utopian']",teenager living leads gang nightly orgies rand...,"[-0.025159629, -0.042870175, -0.008320468, 0.0...","[fiction, literature, thriller]"
2,The text of The Plague is divided into five p...,"['fiction', 'philosophy', 'literature']",text plague divided five town thousands initia...,"[0.0450471, -0.024600938, 0.011781344, -0.0245...","[fiction, literature, utopian]"
3,The novel posits that space around the Milky ...,"['fiction', 'fantasy']",novel posits space around milky way divided co...,"[-0.092494026, -0.055143498, -0.03134751, 0.03...","[fiction, literature, utopian]"
4,"The book tells the story of Paul B�umer, a Ge...","['war', 'nonfiction']",book tells story paul german soldier school ge...,"[-0.05076918, 0.01998265, -0.003516058, 0.0133...","[fiction, literature, nonfiction]"
5,"Ged is a young boy on Gont, one of the larger...","['children', 'fiction', 'biography', 'fantasy']",ged young boy larger islands north archipelago...,"[0.013353534, 0.044568155, -0.0041369703, -0.0...","[adventure, children, fantasy]"
6,"Living on Mars, Deckard is acting as a consul...",['fiction'],living deckard acting consultant movie crew fi...,"[-0.05087902, -0.059044924, -0.080594674, -0.0...","[adventure, documentary, thriller]"
7,Beginning several months after the events in ...,['fiction'],beginning several months events blade deckard ...,"[-0.070230424, -0.04348265, -0.027057825, -0.0...","[fiction, mystery, thriller]"
8,The book of Job tells the story of an extreme...,['religious'],book job tells story extremely righteous man n...,"[-0.06768392, -0.013505768, 0.0075189755, 0.00...","[evil, literature, religious]"
9,The story is told through the eyes of narrato...,"['fiction', 'literature']",story told eyes narrator james named author ce...,"[0.03800554, -0.11618475, -0.0045171985, 0.057...","[fantasy, literature, thriller]"


In [ ]:
for i in range(10):
  print("\n Plot: \n"+corpus['Plot_summary'][i])
  print("\n Genres predicted: \n: ",corpus['multi_label_genre'][i])


 Plot: 
 Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a philosophy. The animals revolt and drive the drunken and irresponsible Mr Jones from the farm, renaming it "Animal Farm". They adopt Seven Commandments of Animal-ism, the most important of which is, "All animals are equal". Snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly. The pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health. Napoleon takes the pups from the farm dogs and trains them privately. Napoleon and Snowball struggle for leadership. When Snowball announces his plans to build a windmill, Napoleon has his dogs chase Snowball away and declares himself lea

In [ ]:
corpus.drop(['Genres'],axis=1,inplace=True)

In [ ]:
corpus.head()

,Plot_summary,clean_plot_summary,passage_embedding,multi_label_genre
0,"Old Major, the old boar on the Manor Farm, ca...",old old boar manor calls animals farm compares...,"[-0.00936466, -0.046432283, 0.04785304, -0.004...","[action, cookery, war]"
1,"Alex, a teenager living in near-future Englan...",teenager living leads gang nightly orgies rand...,"[-0.025159629, -0.042870175, -0.008320468, 0.0...","[fiction, literature, thriller]"
2,The text of The Plague is divided into five p...,text plague divided five town thousands initia...,"[0.0450471, -0.024600938, 0.011781344, -0.0245...","[fiction, literature, utopian]"
3,The novel posits that space around the Milky ...,novel posits space around milky way divided co...,"[-0.092494026, -0.055143498, -0.03134751, 0.03...","[fiction, literature, utopian]"
4,"The book tells the story of Paul B�umer, a Ge...",book tells story paul german soldier school ge...,"[-0.05076918, 0.01998265, -0.003516058, 0.0133...","[fiction, literature, nonfiction]"


Annotating further books using this logic

In [ ]:
corpus2=pd.read_csv("/content/drive/MyDrive/DSF trials/books_training_cleaned.csv")

In [ ]:
corpus2.head()

,Unnamed: 0,chapter
0,0,truth universally single man inpossession good...
1,1,bennet among earliest waited always intended v...
2,2,assistance herfive ask sufficient drawfrom hus...
3,3,jane elizabeth beencautious praise bingley exp...
4,4,within short walk longbourn lived family thebe...


In [ ]:
corpus2['clean_plot_summary']=corpus2['chapter'].apply(lambda x: remove_contractions(str(x)))

corpus2['clean_plot_summary'] =corpus2['clean_plot_summary'].apply(lambda x: clean_dataset(x))
corpus2['clean_plot_summary'] = corpus2['clean_plot_summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (myOwnStopList)]))

In [ ]:
corpus2['passage_embedding']=corpus2['clean_plot_summary'].apply(lambda x: model.encode(x))

In [ ]:
corpus2.head()

,Unnamed: 0,chapter,clean_plot_summary,passage_embedding
0,0,truth universally single man inpossession good...,truth universally single man inpossession good...,"[0.026355945, -0.07240926, 0.10072061, 0.02090..."
1,1,bennet among earliest waited always intended v...,bennet among earliest waited always intended v...,"[-0.007669703, -0.06175118, 0.0328129, 0.05012..."
2,2,assistance herfive ask sufficient drawfrom hus...,assistance herfive ask sufficient drawfrom hus...,"[0.047942333, -0.048436698, 0.071215294, 0.033..."
3,3,jane elizabeth beencautious praise bingley exp...,jane elizabeth beencautious praise bingley exp...,"[-0.01462117, -0.035909534, 0.057234176, 0.054..."
4,4,within short walk longbourn lived family thebe...,within short walk longbourn lived family thebe...,"[0.047342535, -0.069921225, 0.04456482, 0.0206..."


In [ ]:
corpus2['multi_label_genre']=corpus2['passage_embedding'].apply(lambda x: compute_top_3_cosine_similar_genres(x,labels_vectors))

In [ ]:
corpus2.head()

,Unnamed: 0,chapter,clean_plot_summary,passage_embedding,multi_label_genre
0,0,truth universally single man inpossession good...,truth universally single man inpossession good...,"[0.026355945, -0.07240926, 0.10072061, 0.02090...","[comedy, literature, romance]"
1,1,bennet among earliest waited always intended v...,bennet among earliest waited always intended v...,"[-0.007669703, -0.06175118, 0.0328129, 0.05012...","[comedy, cookery, literature]"
2,2,assistance herfive ask sufficient drawfrom hus...,assistance herfive ask sufficient drawfrom hus...,"[0.047942333, -0.048436698, 0.071215294, 0.033...","[children, literature, romance]"
3,3,jane elizabeth beencautious praise bingley exp...,jane elizabeth beencautious praise bingley exp...,"[-0.01462117, -0.035909534, 0.057234176, 0.054...","[literature, sorrow, utopian]"
4,4,within short walk longbourn lived family thebe...,within short walk longbourn lived family thebe...,"[0.047342535, -0.069921225, 0.04456482, 0.0206...","[comedy, romance, sorrow]"


In [ ]:
for i in range(5):
  print("\n Plot: \n"+corpus2['chapter'][i])
  print("\n Genres predicted: \n: ",corpus2['multi_label_genre'][i])


 Plot: 
truth universally single man inpossession good must want little known feelings views man may beon first entering truth wellfixed minds surrounding isconsidered rightful property dear said lady youheard netherfield park let bennet replied returned long andshe told bennet made want know taken cried want tell objection hearing invitation must long says netherfield istaken young man large fortune north came monday chaise four see much delighted agreed take possession servants house bythe end next married single man large five thousand fine thing affect dear replied must know thinking marrying design settling talk likelythat may fall love therefore youmust visit soon see occasion girls may maysend perhaps foras handsome bingley might like youthe best flatter certainly share pretend anything extraordinary woman five ought give overthinking woman often much beauty think must indeed see bingley hecomes engage assure consider think establishment itwould sir william lady lucas aredeterm

concatenating summary data and book data for our training purpose

In [ ]:
df1=corpus[['clean_plot_summary','multi_label_genre']]
df2=corpus2[['clean_plot_summary','multi_label_genre']]

res=pd.concat([df1,df2],ignore_index=True)

print(res.shape)

(13132, 2)


In [ ]:
res.head()

,clean_plot_summary,multi_label_genre
0,old old boar manor calls animals farm compares...,"[action, cookery, war]"
1,teenager living leads gang nightly orgies rand...,"[fiction, literature, thriller]"
2,text plague divided five town thousands initia...,"[fiction, literature, utopian]"
3,novel posits space around milky way divided co...,"[fiction, literature, utopian]"
4,book tells story paul german soldier school ge...,"[fiction, literature, nonfiction]"


Storing preprocessed csv  for future usage

In [ ]:
res.to_csv("/content/drive/MyDrive/DSF trials/preprocessed_annotated_data.csv",index=False)
